<a href="https://colab.research.google.com/github/AlwaysEden/Machine-Learning-HW/blob/main/HW5_kMeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount your drive
from google.colab import drive
drive.mount('/content/drive')

# Code

1.   Load the MNIST dataset as in Homework 1.
Filter the dataset into a subset that contains only the '3' and '9' images.

2.   Implement and run the kMeans algorithm on this subset with k = 2, 3, 5, 10.

3.   Reduce the dimensionality of the raw '3' and '9' images of the subset from 784 to eigenspaces 2, 5 and 10 dimensions (that is, you will have 3 new datasets). Implement and run the kMeans algorithm on these new subsets with k = 2, 3, 5, 10.

In total, you'd run your kMeans 16 times.

In [36]:
import six.moves.cPickle as pickle
import gzip
import os
import numpy as np
import urllib.request

In [37]:
def load_data(dataset):

    if not os.path.isfile(dataset):
        origin = (
            'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        )
        print('Downloading data from %s' % origin)
        urllib.request.urlretrieve(origin, dataset)

    print('Loading data...')

    with gzip.open(dataset, 'rb') as f:
        try:
            train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        except:
            train_set, valid_set, test_set = pickle.load(f)

    all_data_x = np.concatenate([train_set[0], valid_set[0], test_set[0]])
    all_data_y = np.concatenate([train_set[1], valid_set[1], test_set[1]])

    print('... data has been loaded!')
    return all_data_x, all_data_y

In [38]:
def create_k(number_of_k, images):
  k = [None] * number_of_k
  for i in range(number_of_k):
    k_index = np.random.randint(0, len(images))
    k[i] = images[k_index].copy()
  return k

def euclidean_distance(x1, x2):
    distance = np.sqrt(np.sum((x1 - x2) ** 2))
    return distance

def clustering(k, cluster, images):
  for i in range(len(images)):
    distances = [None] * len(k)
    for j in range(len(k)):
      distances[j] = euclidean_distance(images[i], k[j])
    cluster[i] = np.argmin(distances)

def kMeans(num_k, images):
  cluster = [None] * len(images)
  k = create_k(num_k, images) # Assign k randomly
  tolerance = 0.0001 # Check if the mean change
  stop_signal = 0
  while stop_signal < num_k:
    stop_signal = 0
    clustering(k, cluster, images)
    for i in range(num_k):
      cluster_index = np.where(np.array(cluster)==i)[0]
      mean = np.mean(images[cluster_index], axis=0)
      if np.linalg.norm(mean - k[i]) < tolerance:
        stop_signal += 1
      k[i] = mean.copy()

  return cluster.copy()

def showImages(k, cluster, images):
  index = [None] * k
  for i in range(k):
    index[i] = np.where(np.array(cluster)==i)[0]

  for x in range(k):
    for i in index[x]:
      plt.imshow(images[i].reshape(28,28))
      plt.show()
      if i > 50:
        print("***********************************")
        break

def PCA(k, images):
  mean = np.mean(images, axis=0)
  data_center = images - mean
  cov_matrix = np.cov(data_center, rowvar=False)

  eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

  sorted_indices = np.argsort(eigenvalues)[::-1]
  sorted_eigenvalues = eigenvalues[sorted_indices]
  sorted_eigenvectors = eigenvectors[:, sorted_indices]

  principal_components = sorted_eigenvectors[:, :k]

  scaled_images = np.dot(data_center, principal_components)
  return scaled_images

In [39]:
#Load the data into train, validation and test sets
data_x, data_y= load_data('mnist.pkl.gz')

indices_3 = np.where(data_y == 3)[0]
indices_9 = np.where(data_y == 9)[0]

images_3 = data_x[indices_3]
images_9 = data_x[indices_9]

images = np.concatenate([images_3, images_9])
np.random.shuffle(images)

Loading data...
... data has been loaded!


In [34]:

#########################################

# k = 2, dim = 784
cluster_2_784 = kMeans(2, images)

# k = 3, dim = 784
cluster_3_784 = kMeans(3, images)

# k = 5, dim = 784
cluster_5_784 = kMeans(5, images)

# k = 10, dim = 784
cluster_10_784 = kMeans(10, images)

#########################################
scaled_images = PCA(2, images)

# k = 2, dim = 2
cluster_2_2 = kMeans(2, scaled_images)

# k = 3, dim = 2
cluster_3_2 = kMeans(3, scaled_images)

# k = 5, dim = 2
cluster_5_2 = kMeans(5, scaled_images)

# k = 10, dim = 2
cluster_10_2 = kMeans(10, scaled_images)

#########################################
scaled_images = PCA(5, images)

# k = 2, dim = 5
cluster_2_5 = kMeans(2, scaled_images)

# k = 3, dim = 5
cluster_3_5 = kMeans(3, scaled_images)

# k = 5, dim = 5
cluster_5_5 = kMeans(5, scaled_images)

# k = 10, dim = 5
cluster_10_5 = kMeans(10, scaled_images)

#########################################

scaled_images = PCA(10, images)

# k = 2, dim = 10
cluster_2_10 = kMeans(2, scaled_images)

# k = 3, dim = 10
cluster_3_10 = kMeans(3, scaled_images)

# k = 5, dim = 10
cluster_5_10 = kMeans(5, scaled_images)

# k = 10, dim = 10
cluster_10_10 = kMeans(10, scaled_images)

In [ ]:
# Roughly Checking Cluster
#control k and cluster name(cluster_2_784, cluster_3_784, cluster_5_784, cluster_10_784)
'''
print("*****k=2, dim = 784******")
showImages(2, cluster_2_784, images)
print("*****k=3, dim = 784******")
showImages(3, cluster_3_784, images)
print("*****k=5, dim = 784******")
showImages(5, cluster_5_784, images)
print("*****k=10, dim = 784******")
showImages(10, cluster_10_784, images)
''

In [ ]:
# Roughly Checking Cluster
#control k and cluster name(cluster_2_2, cluster_3_2, cluster_5_2, cluster_10_2)
'''
print("*****k=2, dim = 2******")
showImages(2, cluster_2_2, images)
print("*****k=3, dim = 2******")
showImages(3, cluster_3_2, images)
print("*****k=5, dim = 2******")
showImages(5, cluster_5_2, images)
print("*****k=10, dim = 2******")
showImages(10, cluster_10_2, images)
'''

In [ ]:
#Roughly Checking Cluster
#control k and cluster name(cluster_2_5, cluster_3_5, cluster_5_5, cluster_10_5)
'''
print("*****k=2, dim = 2******")
showImages(2, cluster_2_5, images)
print("*****k=3, dim = 2******")
showImages(3, cluster_3_5, images)
print("*****k=5, dim = 2******")
showImages(5, cluster_5_5, images)
print("*****k=10, dim = 2******")
showImages(10, cluster_10_5, images)
'''

In [ ]:
#Roughly Checking Cluster
#control k and cluster name(cluster_2_10, cluster_3_10, cluster_5_10, cluster_10_10)
'''
print("*****k=2, dim = 10******")
showImages(2, cluster_2_10, images)
print("*****k=3, dim = 10******")
showImages(3, cluster_3_10, images)
print("*****k=5, dim = 10******")
showImages(5, cluster_5_10, images)
print("*****k=10, dim = 10******")
showImages(10, cluster_10_10, images)
'''

# ChatGPT prompts

Now solve the same task with ChatGPT with the following prompts:


1.   Can you provide a code to filter the MNIST dataset so that we only have the 9 and 3 samples?
2.   Can you compute the kmeans of this filtered dataset without using any specialized library (coding kmeans from scratch) with k=2,3,5,10?
3.   Can you reduce the dimensionality of the filtered dataset to eigenspaces of dimension 2, 5 and 10 instead of 784?
4.   Can you use the kmeans function from scratch and compute it for the new projected datasets using k=2,3,5 and 10?

**Paste the answer provided and run the code below:**

In [ ]:
def load_data(dataset):

    # Download the MNIST dataset if it is not present
    if not os.path.isfile(dataset):
        origin = (
            'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        )
        print('Downloading data from %s' % origin)
        urllib.request.urlretrieve(origin, dataset)

    print('Loading data...')

    # Load the dataset
    with gzip.open(dataset, 'rb') as f:
        try:
            train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        except:
            train_set, valid_set, test_set = pickle.load(f)

    all_data_x = np.concatenate([train_set[0], valid_set[0], test_set[0]])
    all_data_y = np.concatenate([train_set[1], valid_set[1], test_set[1]])

    print('... data has been loaded!')
    return all_data_x, all_data_y

def filter_mnist(X, y, digits=(3, 9)):
    """
    Filters the MNIST dataset to include only the specified digits.

    :param X: NumPy array or list of images (shape: [number_of_samples, height, width])
    :param y: NumPy array or list of labels (shape: [number_of_samples])
    :param digits: A tuple containing the digits to filter for (default: (3, 9))
    :return: Filtered images and labels for the specified digits
    """
    # Initialize lists to hold the filtered data
    filtered_X = []
    filtered_y = []

    # Iterate through each label in y
    for i in range(len(y)):
        if y[i] in digits:
            filtered_X.append(X[i])
            filtered_y.append(y[i])

    return np.array(filtered_X), np.array(filtered_y)


data_x, data_y= load_data('mnist.pkl.gz')
X_filtered, y_filtered = filter_mnist(data_x, data_y)

Loading data...
... data has been loaded!


In [ ]:
import numpy as np

def initialize_centroids(X, k):
    """ Randomly initialize centroids. """
    random_indices = np.random.choice(X.shape[0], k, replace=False)
    return X[random_indices]

def compute_distances(X, centroids):
    """ Compute the distance from each point to each centroid. """
    distances = np.zeros((X.shape[0], centroids.shape[0]))
    for i in range(centroids.shape[0]):
        distances[:, i] = np.linalg.norm(X - centroids[i], axis=1)
    return distances

def assign_clusters(X, centroids):
    """ Assign each data point to the nearest centroid. """
    distances = compute_distances(X, centroids)
    return np.argmin(distances, axis=1)

def update_centroids(X, labels, k):
    """ Update centroids based on the mean of assigned points. """
    new_centroids = np.zeros((k, X.shape[1]))
    for i in range(k):
        new_centroids[i] = X[labels == i].mean(axis=0)
    return new_centroids

def kmeans(X, k, max_iters=100):
    """ Run the KMeans algorithm. """
    centroids = initialize_centroids(X, k)

    for _ in range(max_iters):
        labels = assign_clusters(X, centroids)
        new_centroids = update_centroids(X, labels, k)

        # Check for convergence
        if np.all(centroids == new_centroids):
            break

        centroids = new_centroids

    return labels, centroids

# Example usage
k_values = [2, 3, 5, 10]

# Assuming X_filtered is your filtered dataset of shape (num_samples, 28*28)
# Reshape images to (num_samples, 28*28) for KMeans
X_flattened = X_filtered.reshape(X_filtered.shape[0], -1)

for k in k_values:
    print(f"\nRunning KMeans with k={k}...")
    labels, centroids = kmeans(X_flattened, k)

    # For demonstration, let's print out the centroids and their corresponding labels
    print(f"Centroids:\n{centroids}")
    print(f"Labels (first 10): {labels[:10]}")



Running KMeans with k=2...
Centroids:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Labels (first 10): [0 1 1 1 0 0 1 1 0 0]

Running KMeans with k=3...
Centroids:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Labels (first 10): [2 1 1 0 2 2 1 0 2 2]

Running KMeans with k=5...
Centroids:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Labels (first 10): [1 2 2 0 3 3 2 0 3 3]

Running KMeans with k=10...
Centroids:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Labels (first 10): [2 5 3 7 4 4 5 7 4 2]


In [ ]:
def standardize_data(X):
    """ Standardize the dataset by centering it (mean = 0). """
    mean = np.mean(X, axis=0)
    X_centered = X - mean
    return X_centered

def compute_covariance_matrix(X):
    """ Compute the covariance matrix of the dataset. """
    return np.cov(X, rowvar=False)

def compute_eigenvalues_and_eigenvectors(cov_matrix):
    """ Compute eigenvalues and eigenvectors of the covariance matrix. """
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)  # Use eigh for symmetric matrix
    return eigenvalues, eigenvectors

def pca(X, k):
    """ Perform PCA and reduce the dataset to k dimensions. """
    X_centered = standardize_data(X)
    cov_matrix = compute_covariance_matrix(X_centered)

    eigenvalues, eigenvectors = compute_eigenvalues_and_eigenvectors(cov_matrix)

    # Sort eigenvalues and corresponding eigenvectors
    sorted_indices = np.argsort(eigenvalues)[::-1]  # Sort in descending order
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]

    # Select the top k eigenvectors
    top_k_eigenvectors = sorted_eigenvectors[:, :k]

    # Project the data onto the new eigenspace
    X_reduced = np.dot(X_centered, top_k_eigenvectors)

    return X_reduced, sorted_eigenvalues[:k]

# Example usage
k_values = [2, 5, 10]

# Assuming X_filtered is your filtered dataset of shape (num_samples, 28*28)
# Reshape images to (num_samples, 28*28) for PCA
X_flattened = X_filtered.reshape(X_filtered.shape[0], -1)

for k in k_values:
    print(f"\nReducing dimensionality to k={k}...")
    X_reduced, eigenvalues = pca(X_flattened, k)

    # Print results
    print(f"Reduced shape: {X_reduced.shape}")
    print(f"Eigenvalues (first {k}): {eigenvalues}")



Reducing dimensionality to k=2...
Reduced shape: (14099, 2)
Eigenvalues (first 2): [7.62428636 3.98411468]

Reducing dimensionality to k=3...
Reduced shape: (14099, 3)
Eigenvalues (first 3): [7.62428636 3.98411468 2.91091582]

Reducing dimensionality to k=5...
Reduced shape: (14099, 5)
Eigenvalues (first 5): [7.62428636 3.98411468 2.91091582 2.3414852  2.17304102]

Reducing dimensionality to k=10...
Reduced shape: (14099, 10)
Eigenvalues (first 10): [7.62428636 3.98411468 2.91091582 2.3414852  2.17304102 1.75331286
 1.26774586 1.11704296 1.09760913 1.0540397 ]


In [ ]:
import numpy as np
import os
import gzip
import pickle
import urllib.request

def load_data(dataset):
    # Download the MNIST dataset if it is not present
    if not os.path.isfile(dataset):
        origin = (
            'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        )
        print('Downloading data from %s' % origin)
        urllib.request.urlretrieve(origin, dataset)

    print('Loading data...')

    # Load the dataset
    with gzip.open(dataset, 'rb') as f:
        try:
            train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        except:
            train_set, valid_set, test_set = pickle.load(f)

    all_data_x = np.concatenate([train_set[0], valid_set[0], test_set[0]])
    all_data_y = np.concatenate([train_set[1], valid_set[1], test_set[1]])

    print('... data has been loaded!')
    return all_data_x, all_data_y

def standardize_data(X):
    """ Standardize the dataset by centering it (mean = 0). """
    mean = np.mean(X, axis=0)
    X_centered = X - mean
    return X_centered

def compute_covariance_matrix(X):
    """ Compute the covariance matrix of the dataset. """
    return np.cov(X, rowvar=False)

def compute_eigenvalues_and_eigenvectors(cov_matrix):
    """ Compute eigenvalues and eigenvectors of the covariance matrix. """
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)  # Use eigh for symmetric matrix
    return eigenvalues, eigenvectors

def pca(X, k):
    """ Perform PCA and reduce the dataset to k dimensions. """
    X_centered = standardize_data(X)
    cov_matrix = compute_covariance_matrix(X_centered)

    eigenvalues, eigenvectors = compute_eigenvalues_and_eigenvectors(cov_matrix)

    # Sort eigenvalues and corresponding eigenvectors
    sorted_indices = np.argsort(eigenvalues)[::-1]  # Sort in descending order
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]

    # Select the top k eigenvectors
    top_k_eigenvectors = sorted_eigenvectors[:, :k]

    # Project the data onto the new eigenspace
    X_reduced = np.dot(X_centered, top_k_eigenvectors)

    return X_reduced, sorted_eigenvalues[:k]

def initialize_centroids(X, k):
    """ Randomly initialize centroids. """
    random_indices = np.random.choice(X.shape[0], k, replace=False)
    return X[random_indices]

def compute_distances(X, centroids):
    """ Compute the distance from each point to each centroid. """
    distances = np.zeros((X.shape[0], centroids.shape[0]))
    for i in range(centroids.shape[0]):
        distances[:, i] = np.linalg.norm(X - centroids[i], axis=1)
    return distances

def assign_clusters(X, centroids):
    """ Assign each data point to the nearest centroid. """
    distances = compute_distances(X, centroids)
    return np.argmin(distances, axis=1)

def update_centroids(X, labels, k):
    """ Update centroids based on the mean of assigned points. """
    new_centroids = np.zeros((k, X.shape[1]))
    for i in range(k):
        new_centroids[i] = X[labels == i].mean(axis=0)
    return new_centroids

def kmeans(X, k, max_iters=100):
    """ Run the KMeans algorithm. """
    centroids = initialize_centroids(X, k)

    for _ in range(max_iters):
        labels = assign_clusters(X, centroids)
        new_centroids = update_centroids(X, labels, k)

        # Check for convergence
        if np.all(centroids == new_centroids):
            break

        centroids = new_centroids

    return labels, centroids

# Load the MNIST dataset
data_file = 'mnist.pkl.gz'
X, y = load_data(data_file)

# Filter the dataset for only digits 3 and 9
mask = np.isin(y, [3, 9])
X_filtered = X[mask]
y_filtered = y[mask]

# Flatten the filtered dataset for PCA
X_flattened = X_filtered.reshape(X_filtered.shape[0], -1)

# Define the k values for KMeans
k_values = [2, 3, 5, 10]
dim_values = [0,2,5,10]
for k in k_values:
  for dim in dim_values:
    if dim == 0:
      labels, centroids = kmeans(X_flattened, k)
      print(f"Running KMeans on reduced dataset with k={k}, dim=784...")
      continue
    print(f"\nReducing dimensionality to k={k} using PCA...")
    X_reduced, eigenvalues = pca(X_flattened, dim)
    labels, centroids = kmeans(X_reduced, k)

    print(f"Running KMeans on reduced dataset with k={k}, dim={dim}...")


    # Print results
    print(f"Centroids:\n{centroids}")
    #print(f"Labels (first 10): {labels[:10]}")


# Report



1.   What are some differences you notice between your code and ChatGPT's? (Name at least 3)
2.   Ask ChatGPT how can you judge if a cluster is correctly computed. Paste the answer here. Do you agree with ChatGPT's statement? Why yes or why not?
3.   Ask ChatGPT how can we know which k is the correct parameter for a kMeans algorithm. Paste the answer here. Do you agree with ChatGPT's statement? Why yes or why not?
4.   Conclude with some thoughts and things you learned from this homework.

<h1>Answer</h1>
1.

- Clustering End condition: ChatGPT made the kMeans code iterate up to 100 times, but I made the kMeans code iterate until the error is less than 0.001. I think this is a trade-off of time and accuracy for more accurate clustering, so ChatGPT is always faster to compute.
- Filter MNIST: ChatGPT splits the data into 3 and 9 images using np.isin(). It can help me find the data easily.
- Code Structure: I made the code structure for readability. However, ChatGPT made the code structure for efficiency.
2.

I generally agree with these idea. First of all, Within-cluster distance and Between-cluster is a core concepts of inner and outter cluster. When theses concepts are considered together, we can get a better clustering. Second, Elbow method and Visualization are a good approach. Elbow method is a mathematic way to prove it and the concept is very clear and understandable. And visualization is very intuitive. When we use lower-dimensional data, visualization is useful for us to understand it in one glance. However, Cluster interpretability and Silhouette score are carefully considered. Cluster interpretability can be used easily when the data is labeled. But, If the data is labled, better approaches exist. And when the data is not labled, it is based on the domain knowledge. If so, I would rather use a different algorithm. In Silhouette score, we need to think of the reason why all cluster should be similar. In my opinion, if the data is many different, the cluster is also many different.

3.

The methods which decide if k is correct are very similar to the methods which decided if cluster is correctly computed. These methods consist of mathematical approaches, approaches in terms of data, Heuristic approaches, and etc. I generally agree with these ideas the same way I agree with the cluster problem.

4.

- Eventully, Algorithm. General algorithms in CS are the methods to solve some problem using data structure and logical thinking. kMeans is also algorithm and it's just data with dimensions, not formal data like a data structure.
- Understanding Dimension. MNIST image data is [70000, 784]. When I tied to split the data into iamge 3 and 9, I didn't know what is 3 and 9 data. After understanding image data and dimension, I could solve this problem.
- Understanding narray library. If it is C, many things are impossible. But, I was able to calculate some complicated problem easily thanks to narray such as np.argmin and np.where.


## ChatGPT's Answer about Q2

Within-cluster distance (compactness): A well-defined cluster should have minimal variance within itself. You can measure how close the data points in each cluster are to the cluster centroid (e.g., using sum of squared distances). The smaller the distance, the more compact the cluster.

Between-cluster separation: Clusters should be well-separated from each other. You can check how far the centroids are from each other. Larger distances between centroids suggest better separation between clusters.

Silhouette score: This metric calculates how similar a data point is to its own cluster compared to other clusters. Scores range from -1 to 1, where higher values indicate better clustering.

Cluster interpretability: If the data is labeled, compare the cluster assignments to the actual labels. The clusters should correspond well to the natural groups in the data (e.g., digits 3 and 9 in the MNIST dataset). For unsupervised data, interpretability could be based on domain knowledge.

Elbow method: Plot the sum of squared distances from points to their assigned cluster centers for different numbers of clusters (k). The point at which adding more clusters doesn’t significantly reduce the sum of squared distances indicates the "optimal" number of clusters.

Visualization (for 2D or 3D data): For lower-dimensional data, visual inspection can be useful. You should be able to see clear separations between clusters, and points should group around centroids.

## ChatGPT's Answer about Q3
Determining the correct value of \( k \) (the number of clusters) for a KMeans algorithm is not always straightforward. Here are several methods and guidelines commonly used to identify the optimal \( k \):

### 1. **Elbow Method**
   - **How it works**: Plot the sum of squared distances (also called inertia or distortion) from each point to its assigned centroid as a function of \( k \).
   - **What to look for**: As \( k \) increases, the within-cluster variance decreases. The "elbow" of the plot is the point where the decrease in variance starts to slow down significantly, indicating diminishing returns for adding more clusters. This point is considered a good estimate for the optimal \( k \).
   - **Why it works**: It identifies a balance between minimizing the number of clusters and maximizing the compactness of the clusters.

### 2. **Silhouette Score**
   - **How it works**: The silhouette score measures how similar a data point is to its own cluster (cohesion) compared to other clusters (separation). It ranges from -1 to 1.
   - **What to look for**: A silhouette score close to 1 indicates well-separated and compact clusters. A score near 0 means the clusters are overlapping, while negative values suggest wrong cluster assignments.
   - **Why it works**: It balances within-cluster compactness and separation between clusters, helping find a \( k \) that yields well-defined clusters.

### 3. **Gap Statistic**
   - **How it works**: The gap statistic compares the within-cluster dispersion for different values of \( k \) with that expected under a null reference distribution of the data (random distribution). The larger the gap, the better the clustering is compared to random noise.
   - **What to look for**: Select \( k \) where the gap statistic is largest.
   - **Why it works**: It provides a more statistically grounded method to select \( k \) and compares actual clusters with clusters that might arise from random data.

### 4. **Cross-Validation**
   - **How it works**: Use cross-validation techniques, splitting the data into training and validation sets, and evaluate how the KMeans algorithm performs for different values of \( k \).
   - **What to look for**: Choose the \( k \) that performs best on the validation set (e.g., minimizes within-cluster variance or maximizes a specific metric).
   - **Why it works**: This method ensures that the chosen \( k \) generalizes well to unseen data, avoiding overfitting to noise or random variations.

### 5. **BIC/AIC (Bayesian/ Akaike Information Criterion)**
   - **How it works**: These criteria add penalties for model complexity (such as increasing \( k \)) to the likelihood of the model.
   - **What to look for**: Select the value of \( k \) that minimizes either the BIC or AIC.
   - **Why it works**: It accounts for both the goodness of fit and the simplicity of the model, helping to avoid overfitting by choosing too many clusters.

### 6. **Domain Knowledge or Heuristic**
   - **How it works**: In some cases, you might have prior knowledge or expectations about the number of clusters. For instance, if you’re clustering types of customers in marketing, there may be a natural grouping based on demographics or buying behavior.
   - **What to look for**: Use a combination of domain knowledge and clustering results to select \( k \).
   - **Why it works**: Domain knowledge helps ensure that the clusters are meaningful in the context of the problem.

### 7. **Davies-Bouldin Index**
   - **How it works**: This metric evaluates how well clusters are separated by measuring the ratio of within-cluster scatter to between-cluster separation.
   - **What to look for**: A lower Davies-Bouldin Index indicates better clustering. You can use this metric to select the optimal \( k \).
   - **Why it works**: It balances intra-cluster compactness and inter-cluster distance, helping to find well-separated clusters.

### 8. **Dunn Index**
   - **How it works**: The Dunn Index is the ratio of the minimum inter-cluster distance to the maximum intra-cluster distance.
   - **What to look for**: A higher Dunn Index indicates better cluster separation. Use this to select \( k \) where the index is maximized.
   - **Why it works**: It seeks a balance between tight clusters and large separation between clusters.

### Conclusion:
There is no single correct answer to choosing \( k \) for KMeans, but methods like the **Elbow Method**, **Silhouette Score**, and **Gap Statistic** are commonly used for a more objective assessment. Domain knowledge also plays a crucial role in interpreting the clustering results, ensuring that the clusters are meaningful for the specific context.